In [3]:
# load training data
import pandas as pd
print 'Loading trainig data'
train_data = pd.read_csv('training.csv')

Loading trainig data


In [53]:
from sklearn.model_selection import train_test_split
# asign sets and make splits
y       = pd.DataFrame(train_data['Label'])
X       = train_data.iloc[:,:32]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# change strings to integers
y_train['Label'] = y_train['Label'].map({'b':0, 's':1})
y_test['Label'] = y_test['Label'].map({'b':0, 's':1})
# put weights on separate variables
w_train = pd.DataFrame(X_train['Weight'])
w_test  = pd.DataFrame(X_test['Weight'])
# conserve id's in case we use them in the future
id_train = pd.DataFrame(X_train['EventId'])
id_test  = pd.DataFrame(X_test['EventId'])
# leave only the feature vector on the X's
X_train = X_train.iloc[:,1:31]
X_test  = X_test.iloc[:,1:31]

/home/jose/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jose/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [54]:
# train the model
from sklearn.ensemble import GradientBoostingClassifier as GBC
clf = GBC(n_estimators=50, max_depth=5, min_samples_leaf=200, max_features=10, verbose=1)
clf.fit(X_train,y_train)

      Iter       Train Loss   Remaining Time 
         1           1.2188            1.21m
         2           1.1609            1.23m
         3           1.1203            1.20m
         4           1.0791            1.06m
         5           1.0425           57.93s
         6           1.0132           54.17s
         7           0.9844           51.47s
         8           0.9614           48.50s
         9           0.9445           46.32s
        10           0.9246           44.31s
        20           0.8192           28.45s
        30           0.7762           17.87s
        40           0.7537            8.64s
        50           0.7399            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=10, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=200,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=50, presort='auto', random_state=None,
              subsample=1.0, verbose=1, warm_start=False)

In [57]:
# predictions from training set
y_train_pred = clf.predict(X_train)
# probabilities from train/test
prob_train_pred = clf.predict_proba(X_train)
prob_test_pred  = clf.predict_proba(X_test)
# get second column (first column has inverse probabilities)
prob_train_pred = prob_train_pred[:,1]
prob_test_pred  = prob_test_pred[:,1]

array([ 0.76656025,  0.08582114,  0.71906192, ...,  0.06783308,
        0.07816643,  0.15369901])

In [97]:
import numpy as np
# the 85th percentile is the value below which 85% of the observations may be found (wikipedia)
threshold = np.percentile(prob_train_pred, 85)
# the threshold displayed 0.78445009849346203
y_train_pred = prob_train_pred > threshold 
y_test_pred  = prob_test_pred  > threshold
print y_train_pred
print y_test_pred

[False False False ..., False False False]
[False False False ..., False False False]


In [189]:
# compute true positive/negatives and scale the weights according to the train/test split
TP_train = w_train.values * (y_train == 1.0)*(1.0/0.67)
TN_train = w_train.values * (y_train == 0.0)*(1.0/0.67)
TP_test  = w_test.values  * (y_test  == 1.0)*(1.0/0.33)
TN_test  = w_test.values  * (y_test  == 0.0)*(1.0/0.33)
TP_train.values.flatten()

array([ 0.0278151 ,  0.        ,  0.00224284, ...,  0.        ,
        0.        ,  0.        ])

In [190]:
y_train_pred

array([False, False, False, ..., False, False, False], dtype=bool)

In [191]:
# compute s's and b's to be used in the AMS calculation accordin to the documentation
# ussage of the indicator funcion
s_train = sum(TP_train.values.flatten() * (y_train_pred == 1.0))
b_train = sum(TN_train.values.flatten() * (y_train_pred == 1.0))
s_test = sum(TP_test.values.flatten()   * (y_test_pred  == 1.0))
b_test = sum(TN_test.values.flatten()   * (y_test_pred  == 1.0))

In [192]:
# calculate the AMS score
import math
def AMS_score(s,b): return  math.sqrt (2.*( (s + b + 10.)*math.log(1.+s/(b+10.))-s))
print 'AMS score with training sample: {}'.format(AMS_score(s_train,b_train))
print 'AMS score with testing sample: {}:'.format(AMS_score(s_test,b_test))

AMS score with training sample: 3.49929238197
AMS score with testing sample: 3.42973369564:


In [121]:
# load testing data to perform the submission predictions
test_data = pd.read_csv('test.csv')

In [144]:
# store the feature columns and make the predictions
X_t = test_data.iloc[:,1:31]
y_prob_t = clf.predict_proba(X_t)
y_prob_t = y_prob_t[:,1]

In [176]:
# get probabilities into a dataframe, stablish values above the threshold and set id's
probs = pd.DataFrame(y_prob_t,columns=['prob'])
y_t_cleaned = y_prob_t > threshold
labels = pd.DataFrame(y_t_cleaned, columns=['label'])
submission = pd.DataFrame([test_data['EventId'], probs['prob'], labels['label']]).T

In [181]:
# we first sort the data based on the probabilities to later assign the rank numeration incrementally
sort_submission = submission.sort(['prob'],ascending=True)
sort_submission.head()

/home/jose/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,EventId,prob,label
122596,472596.0,0.016690,0.0
370548,720548.0,0.017145,0.0
380878,730878.0,0.017216,0.0
103392,453392.0,0.017227,0.0
96821,446821.0,0.017293,0.0


In [182]:
# assign rank numeration incrementally
ranks = [i+1 for i in range(0, X_t.shape[0])]
sort_submission['prob']    = ranks
# return to original labels
sort_submission['label']   = sort_submission['label'].map({1:'s', 0:'b'})
# return id's to original format
sort_submission['EventId'] = sort_submission['EventId'].astype(int)
sort_submission.head()

,EventId,prob,label
122596,472596,1,b
370548,720548,2,b
380878,730878,3,b
103392,453392,4,b
96821,446821,5,b


In [183]:
# set columns correct naming
id_sort = sort_submission.sort(['EventId'], ascending=True)
id_sort.columns=['EventId','RankOrder','Class']
id_sort.tail()

/home/jose/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,EventId,RankOrder,Class
549995,899995,159564,b
549996,899996,375404,b
549997,899997,124026,b
549998,899998,383427,b
549999,899999,287662,b


In [184]:
# generate submission file
'''submission = pd.DataFrame({
        "EventId": id_sort['EventId'],
        "RankOrder":id_sort['RankOrder'],
        "Class": id_sort['Class']
    })
'''
# this submission gets to place 882/1785
id_sort.to_csv('higgsSubmission1.csv', index=False)